<ul class="breadcrumb">
  <li><a href="1.2.Multivariate_Linear_Mixed_Effects_Model.ipynb">Multivariate Basics</a></li>
  <li><a href="2.2.Multivariate_Linear_Additive_Genetic_Model.ipynb">Multivariate Additive Genetic Model</a></li> 
  <li><a href="3.2.Multivariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Multivariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Additive Genetic Model</font><br> 
  <font size="5" face="Georgia">(with Maternal Effects)</font> 

</div>

In [2]:
using DataFrames,CSV,JWAS,JWAS.Datasets

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">I. Multivariate Linear Additive Genetic Model</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [3]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### phenotypes

In [4]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### pedigree

In [5]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


In [6]:
data = CSV.read(phenofile,delim = ',',header=true,null="NA");

In [7]:
data

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [8]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [9]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";

In [10]:
model    = build_model(model_equations,R);

In [11]:
set_covariate(model,"age");

#### set variables as random variables

* random variables whose covariance matrices are numerator relationship matrix 

In [12]:
ped = get_pedigree(pedfile);

coding pedigree... 100%|████████████████████████████████| Time: 0:00:01

Finished!



calculating inbreeding... 100%|█████████████████████████| Time: 0:00:00


In [13]:
set_random(model,"Animal", ped,G)

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [14]:
data

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


In [15]:
out = runMCMC(model,data,chain_length=1000,printout_frequency=500,output_samples_frequency=10);

MCMC Information:

methods                        conventional (no markers)
chain_length                                   1000
starting_value                                false
printout_frequency                              500
output_samples_frequency                         10
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000





running MCMC for conventional (no markers)...  0%|      |  ETA: 0:07:46


Posterior means at iteration: 500

running MCMC for conventional (no markers)... 23%|█     |  ETA: 0:00:02


Residual covariance matrix: 
[129.784 33.218; 33.218 9.30655]
Polygenic effects covariance matrix 
[33.6449 1.67446; 1.67446 1.51695]



running MCMC for conventional (no markers)... 50%|███   |  ETA: 0:00:01


Posterior means at iteration: 1000
Residual covariance matrix: 
[110.944 27.6372; 27.6372 7.6247]
Polygenic effects covariance matrix 
[36.4908 3.77872; 3.77872 2.2476]



running MCMC for conventional (no markers)...100%|██████| Time: 0:00:01


In [16]:
out["MCMC samples for residual covariance matrix"]

4×100 Array{Float64,2}:
 366.773   70.8631   158.221    96.6215   …  212.373   193.757   132.726 
 105.237   18.9708    29.3475   22.5761       52.3629   52.3978   35.8164
 105.237   18.9708    29.3475   22.5761       52.3629   52.3978   35.8164
  30.3625   5.59425    5.86322   5.49077      13.0509   14.5518    9.8129

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">II. The Multivariate Linear Additive Genetic Model with Maternal Effects</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [17]:
phenofile = Datasets.dataset("testMT","maternal.txt")
data = CSV.read(phenofile,delim = ',',header=true,null="NA");

<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [18]:
model_equations = "BW = intercept + age + sex + Animal+ dam;
                   CW = intercept + age + sex + Animal";

In [19]:
model = build_model(model_equations,R);
set_covariate(model,"age");

In [20]:
# order is BW:Animal, BW:Dam, CW: Animal
G0 = [5   1    0.1
      1   1    0.01
      0.1 0.01 0.5] 
set_random(model,"Animal dam", ped,G0)

INFO: dam is not found in model equation 2.


In [21]:
randomStr = "Animal dam"
pedTrmVec = split(randomStr," ",keep=false)  # "animal animal*age"

2-element Array{SubString{String},1}:
 "Animal"
 "dam"   

In [22]:
out=runMCMC(model,data,chain_length=1000,printout_frequency=500);

MCMC Information:

methods                        conventional (no markers)
chain_length                                   1000
starting_value                                false
printout_frequency                              500
output_samples_frequency                          0
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000




Posterior means at iteration: 500
Residual covariance matrix: 
[137.968 28.5854; 28.5854 6.6699]
Polygenic effects covariance matrix 
[5.76356 1.11352 0.337443; 1.11352 0.905174 0.071411; 0.337443 0.071411 0.494804]



running MCMC for conventional (no markers)... 25%|██    |  ETA: 0:00:00


Posterior means at iteration: 1000
Residual covariance matrix: 
[133.266 29.5294; 29.5294 7.20693]
Polygenic effects covariance matrix 
[5.33444 1.01254 0.256796; 1.01254 0.946601 0.061943; 0.256796 0.061943 0.474664]



running MCMC for conventional (no markers)...100%|██████| Time: 0:00:00


In [23]:
keys(out)

Base.KeyIterator for a Dict{Any,Any} with 3 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "Posterior mean of residual covariance matrix"
  "Posterior mean of location parameters"